# Reading MERRA-2 Data from a Downloaded netCDF4 File

In [ ]:
import xarray as xr
from matplotlib import pyplot

## Using a Downloaded netCDF4 File

In [ ]:
ds = xr.open_dataset('/home/arthur.endsley/Downloads/MERRA2_400.statD_2d_slv_Nx.20230701.nc4')
ds

In [ ]:
ds['T2MMIN']

In [ ]:
ds['T2MMIN'].attrs

In [ ]:
ds['T2MMIN'].plot()

In [ ]:
ds['T2MMIN'].data

In [ ]:
ds['T2MMIN'].data.shape

In [ ]:
ds['T2MMIN'].sel(lat = -90, lon = -180)

In [ ]:
ds['T2MMIN'].sel(lat = -90, lon = -180).values

What's the minimum temperature in Algiers? Note that we must specify a `method` here because the coordinates of Algiers don't exactly match the coordinates of each grid cell's center; i.e., we must ask for a nearest-neighbor interpolation.

In [ ]:
ds['T2MMIN'].sel(lat = 36.754, lon = 3.059, method = 'nearest').values

## TODO Too early for this example

In [ ]:
ds['T2MMIN'].sel(lat = slice(30, 40), lon = slice(0, 6)).plot()